In [5]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3
import pandas as pd
import numpy as np

sm_boto3=boto3.client("sagemaker")
sess=sagemaker.Session()
region=sess.boto_session.region_name
bucket='mobbucketsagemakertest38'
print('Using bucket :' + bucket)


Using bucket :mobbucketsagemakertest38


In [25]:
data=pd.read_csv('C:\\Users\\etulyon1\\Documents\\aws\\dataset\\mobil_train.csv')
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [26]:
y=data.price_range.values
np.unique(y)

array([0, 1, 2, 3], dtype=int64)

In [27]:
data.isna().sum()

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report



In [29]:

x_train,x_test=train_test_split(data,test_size=0.2)


In [30]:
X_train=pd.DataFrame(x_train)
X_train.shape
X_train.to_csv('trainV1.csv',index=False)
X_test=pd.DataFrame(x_test)
X_test.to_csv('testV1.csv',index=False)

In [6]:
prefix="sagemaker/mobil_classification/sklearncontainer"
trainpath=  sess.upload_data(
    path='trainV1.csv', bucket=bucket, key_prefix=prefix
)
testpath=sess.upload_data(
    path='testV1.csv', bucket=bucket, key_prefix=prefix
)
print(trainpath)

s3://mobbucketsagemakertest38/sagemaker/mobil_classification/sklearncontainer/trainV1.csv


In [82]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,precision_score
from sklearn.model_selection import GridSearchCV
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy  as np 
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
if __name__=="__main__":

    print('[INFO] Extracting Arguments')
    parser=argparse.ArgumentParser()
    # hyperparam send by the client and passed as command line
    parser.add_argument('--n_estimator', type=int,default=200)
    parser.add_argument('--random_state',type=int,default=0)
    #Data,model,output
    parser.add_argument('--model-dir',type=str,default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train',type=str,default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--test',type=str,default=os.environ.get('SM_CHANNEL_TEST'))
    parser.add_argument('--train-file',type=str,default='trainV1.csv')
    parser.add_argument('--test-file',type=str,default='testV1.csv')

    args,_=parser.parse_known_args()

    train_df=pd.read_csv(os.path.join(args.train,args.train_file))
    test_df=pd.read_csv(os.path.join(args.test,args.test_file))
    features=list(train_df.columns)
    label=features.pop(-1)

    print('testing the model')

    X_train=train_df[features]
    X_test=test_df[features]
    y_train=train_df[label]
    y_test=test_df[label]

    print('Trainig the model')
#Definission des parametre a tester

    param_grid={

        'n_estimators': [ 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }


    model=RandomForestClassifier(random_state=args.random_state)
    # entrainement de la grille trouver les meilleures hyperparametres

    grid=GridSearchCV(model,param_grid,cv=4,scoring='accuracy',n_jobs=-1 )

    grid.fit(X_train,y_train)
    model=grid.best_estimator_  
    print(model)
    model.fit(X_train,y_train)

    model_path=os.path.join(args.model_dir,'model.joblib')
    joblib.dump(model,model_path)

    print('model at '+ model_path)

    y_pred_test=model.predict(X_test)
    test_acc=accuracy_score(y_test,y_pred_test)
    test_rep=classification_report(y_test,y_pred_test)

    print('accuracy : ',test_acc)

    print('class_report : ',test_rep)



Overwriting script.py


In [2]:
from sagemaker.sklearn.estimator import SKLearn
FRAMEWORK_VERSION="0.23-1"
sklearn_estimator=SKLearn(
    entry_point="script.py",
    role='arn:aws:iam::850995554854:role/sagemakertuto',
    instance_count=1,
    instance_type='ml.m5.large',
    framework_version=FRAMEWORK_VERSION,
    base_job_name='RF-custom-sklearn',
    hyperparameters={
        
        'random_state':0,

    },
    enable_managed_spot_training=True,
    use_spot_instance=True
   

)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\etulyon1\AppData\Local\sagemaker\sagemaker\config.yaml


In [7]:
#launch r=training job
sklearn_estimator.fit({'train':trainpath,'test': testpath},wait=True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-11-08-15-32-47-424
2024-11-08 15:32:50 Starting - Starting the training job...
2024-11-08 15:33:05 Starting - Preparing the instances for training...
2024-11-08 15:33:54 Downloading - Downloading the training image......
2024-11-08 15:34:35 Training - Training image download completed. Training in progress...2024-11-08 15:34:40,250 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-08 15:34:40,253 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,289 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-08 15:34:40,467 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,479 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,490 sagemaker-training-toolkit INFO     No GPUs detected (normal if

In [8]:
sklearn_estimator.latest_training_job.wait(logs=None)
artifact=sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name)["ModelArtifacts"]['S3ModelArtifacts']

print('Model artifact persisted at '+ artifact)

2024-11-08 15:37:34 Starting - Preparing the instances for training
2024-11-08 15:37:34 Downloading - Downloading the training image
2024-11-08 15:37:34 Training - Training image download completed. Training in progress.
2024-11-08 15:37:34 Uploading - Uploading generated training model
2024-11-08 15:37:34 Completed - Training job completed2024-11-08 15:34:40,250 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-08 15:34:40,253 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,289 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-08 15:34:40,467 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,479 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40,490 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-08 15:34:40

In [13]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime,strftime

model_name='Custom-sklearn-model-' + strftime('%Y-%m-%d-%H-%M-%S',gmtime())
model=SKLearnModel(
    name= model_name,
    model_data=artifact,
    role='arn:aws:iam::850995554854:role/sagemakertuto',
    entry_point='script.py',
    framework_version=FRAMEWORK_VERSION,
)

In [20]:
#Endpoint deployment
endpoint_name='Custome-sklearn-model-' + strftime('%Y-%m-%d-%H-%M-%S',gmtime())

print('EndpointName={}'.format(endpoint_name))

predictor=model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
)

EndpointName=Custome-sklearn-model-2024-11-08-15-59-24
INFO:sagemaker:Creating model with name: Custom-sklearn-model-2024-11-08-15-42-23
INFO:sagemaker:Creating endpoint-config with name Custome-sklearn-model-2024-11-08-15-59-24
INFO:sagemaker:Creating endpoint with name Custome-sklearn-model-2024-11-08-15-59-24
-----!

In [42]:
X_test[features][0:2].values.tolist()

[[1300.0,
  1.0,
  0.5,
  0.0,
  5.0,
  0.0,
  62.0,
  0.1,
  156.0,
  6.0,
  7.0,
  219.0,
  1037.0,
  874.0,
  15.0,
  3.0,
  10.0,
  1.0,
  1.0,
  0.0],
 [1949.0,
  0.0,
  1.3,
  1.0,
  0.0,
  0.0,
  23.0,
  0.1,
  101.0,
  5.0,
  1.0,
  839.0,
  1136.0,
  2478.0,
  14.0,
  8.0,
  20.0,
  1.0,
  0.0,
  0.0]]

In [46]:
print(predictor.predict(X_test[features][0:2].values.tolist()))
print(X_test.iloc[:2,-1])

[0 3]
1187    0
177     3
Name: price_range, dtype: int64


In [48]:
sm_boto3.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'a916f48a-20e3-4b05-a889-65e63d36c398',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a916f48a-20e3-4b05-a889-65e63d36c398',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 08 Nov 2024 18:21:16 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}